<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
# Ensure the right version of Tensorflow is installed.
!pip freeze | grep tensorflow==2.1

In [3]:
# change these to try this notebook out
BUCKET = 'advanced-ml-on-tf'
PROJECT = 'qwiklabs-gcp-00-12ca0c6461c9'
REGION = 'us-central1'

In [4]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [5]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [6]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [7]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,1088037545023002395,362125
1,3408502330831153141,323970
2,-9068386407968572094,330863
3,7186614341837170520,346919
4,6749419160304400104,333076


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [8]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13561 examples in the train dataset and 3334 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [9]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.054792,False,20,1,39.0,3095933535584005890
1,7.336984,False,22,1,43.0,3095933535584005890
2,8.249698,True,24,1,39.0,3095933535584005890
3,6.849762,True,26,1,40.0,3095933535584005890
4,6.913697,False,31,1,38.0,3095933535584005890


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [10]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13552.000000,13561.000000,13561.000000,13466.00000,1.356100e+04
mean,7.221815,27.456382,1.036354,38.59201,2.602463e+17
std,1.349695,6.142761,0.196405,2.59973,5.201075e+18
min,0.500449,13.000000,1.000000,18.00000,-9.183606e+18
25%,6.563162,23.000000,1.000000,38.00000,-3.340563e+18
50%,7.312733,27.000000,1.000000,39.00000,-3.280124e+17
75%,8.062305,32.000000,1.000000,40.00000,4.331750e+18
max,13.406310,50.000000,3.000000,47.00000,8.599690e+18


In [11]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [12]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.054792,False,20,Single(1),39.0,3095933535584005890
1,7.336984,False,22,Single(1),43.0,3095933535584005890
2,8.249698,True,24,Single(1),39.0,3095933535584005890
3,6.849762,True,26,Single(1),40.0,3095933535584005890
4,6.913697,False,31,Single(1),38.0,3095933535584005890


In [13]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13556,1.812200,Unknown,34,Single(1),24.0,-774501970389208065
13557,3.500941,Unknown,22,Single(1),31.0,-774501970389208065
13558,8.487797,Unknown,27,Single(1),41.0,-774501970389208065
13559,7.625790,Unknown,35,Single(1),38.0,-774501970389208065
13560,8.540708,Unknown,17,Single(1),38.0,-774501970389208065


In [14]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26916.000000,26916.000000,26916.000000,2.691600e+04
mean,7.223460,27.459875,38.594813,2.582267e+17
std,1.348532,6.139616,2.589702,5.199251e+18
min,0.500449,13.000000,18.000000,-9.183606e+18
25%,6.563162,23.000000,38.000000,-3.340563e+18
50%,7.312733,27.000000,39.000000,-3.280124e+17
75%,8.062305,32.000000,40.000000,4.331750e+18
max,13.406310,50.000000,47.000000,8.599690e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [15]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [16]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   6602 eval.csv
  26916 train.csv
  33518 total
==> eval.csv <==
6.686620406459999,False,30,Single(1),39.0,1088037545023002395
7.8484565272,True,18,Single(1),38.0,-6244544205302024223
8.54732189774,False,30,Single(1),39.0,-6782146986770280327
9.37405538024,True,22,Single(1),40.0,-1639186255933990135
6.9996768185,False,42,Single(1),39.0,2246942437170405963
6.0627122049999995,True,31,Single(1),41.0,-4740473290291881219
7.50012615324,True,40,Single(1),39.0,-1639186255933990135
5.62619692624,True,19,Single(1),41.0,-1866590652208008467
8.68841774542,False,26,Single(1),37.0,2246942437170405963
5.8753192823,False,32,Single(1),40.0,-1891060869255459203

==> train.csv <==
7.054792384,False,20,Single(1),39.0,3095933535584005890
7.33698407936,False,22,Single(1),43.0,3095933535584005890
8.24969784404,True,24,Single(1),39.0,3095933535584005890
6.84976248034,True,26,Single(1),40.0,3095933535584005890
6.91369653632,False,31,Single(1),38.0,3095933535584005890
2.76900601072,False,38,Single(1),30.0,309

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License